In [7]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F


train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [8]:
# img = torch.squeeze(train_dataset[0][0])
# fig, ax = plt.subplots(nrows=1, ncols=1)
# ax.imshow(img, cmap=plt.cm.gray)
%config Completer.use_jedi = False


In [22]:
#Dataloader 
batch_size = 20
train_loader = DataLoader(
                dataset= train_dataset,
                batch_size=batch_size,
                shuffle=False) 
test_loader = DataLoader(
                dataset = test_dataset,
                batch_size=batch_size,
                shuffle=True)

In [168]:
#setting dtype of the tensors
class NeuralNet():
    def __init__(self, LR, batch_size):
        self.layer1 = np.random.uniform(-0.5, 0.5, (784, 64)).astype(np.float64)
        self.layer2 = np.random.uniform(-0.5, 0.5, (64, 10)).astype(np.float64)
        self.LR = LR
        self.batch_size = batch_size
        
    def softmax(self, x):
        denominators = torch.sum(np.exp(x), 1)
        for i in range(len(x)):
            x[i] = np.divide(x[i], denominators[i])
        return x
    
    def non_linearity(self, x):
        return 1/(1 + np.exp(-x))
        
    def forward(self, x):
        x = torch.squeeze(x).view(-1, 784)
        self.input = x.clone()

        x = x @ self.layer1
        x = self.non_linearity(x)
        
        self.z1 = x.clone()
        x = x @ self.layer2
        
        x = self.softmax(x)
        
        self.softmax_result = x.clone().float().T
        return x
    
    def der_sigmoid(self, x):
        return (x @ (1 - x).T).float()
        
    def backpass(self, loss, outputs, gt):
        d_loss_wrt_preds = (-2/torch.numel(outputs)) * (gt - outputs)
        
        delta_w1 = self.input.T @ self.der_sigmoid(self.z1) @ d_loss_wrt_preds.float() @ self.der_softmax() @ self.layer2.T 
        delta_w2 = self.z1.T.float() @ d_loss_wrt_preds.float() @ self.der_softmax().float() 
        
        self.layer1 = torch.Tensor(self.layer1).float() - self.LR * delta_w1.float()
        self.layer2 = torch.Tensor(self.layer2) - self.LR * delta_w2

    
    def der_softmax(self):
        softmax_d = torch.empty(self.batch_size, len(self.softmax_result), len(self.softmax_result))
        
        for elem in range(self.batch_size):    
            for i in range(softmax_d.shape[1]):
                for j in range(softmax_d.shape[2]):
                    if i == j:
                        softmax_d[elem, i, j] = self.softmax_result.T[elem, i] * (1 - self.softmax_result.T[elem, j])
                    
                    else:
                        softmax_d[elem, i, j] = -1 * self.softmax_result.T[elem, i] * self.softmax_result.T[elem, j]
        return torch.sum(softmax_d, 0).float()
    
    def one_hot_encoding(self, y):
        return F.one_hot(y, num_classes=10)
    
    def mean_sq_error(self, x, y):
        error = torch.sum((x - y) **2) / torch.numel(x)
        return error 

In [ ]:
# Training
def training(EPOCHS):
    total_loss = 0.0
    for epoch in range(EPOCHS):
        for x,y in train_loader:
            
            outputs = net.forward(x)
            gt = net.one_hot_encoding(y)
            batch_loss = net.mean_sq_error(outputs, gt)
#             print(batch_loss)
            net.backpass(batch_loss, outputs, gt)

        total_loss = total_loss + batch_loss 
        print(f"Loss for EPOCH={epoch} is {total_loss}")

# Testing
def testing(test_loader, net):
    for x, y in test_loader:
        output = net.forward(x)
        prediction = torch.argmax(output)
        print(y, prediction)
        break

LR = 0.01
net = NeuralNet(LR, batch_size)
training(5)

/tmp/ipykernel_20286/1028899167.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))


Loss for EPOCH=0 is 0.1007891595363617
Loss for EPOCH=1 is 0.20118889212608337
Loss for EPOCH=2 is 0.30145588517189026
